In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

In [2]:
needed_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

res = requests.get("https://www.themoviedb.org/movie", headers=needed_headers)
res.status_code
# An exit status of 200 means OK 


200

In [5]:
# Creating an instance of the BeautifulSoup class
# also im going to be using lxml instead of the conventional html.parser lets see how that goes
soup = BeautifulSoup(res.text, "lxml")

# won't call soup for my own sanity

In [7]:
web_content = soup.get_text().strip()
# using .strip() right now gets rid of trailing whitespaces but not leading??? wtf? will figure out
print(web_content[:15])

Popular Movies 


### Extracting the title of the web page using various methods:

In [8]:
# Extracting title
soup.title

<title>Popular Movies — The Movie Database (TMDB)</title>

In [9]:
# Another method of fetching the title
soup.select("title")[0].getText()

'Popular Movies — The Movie Database (TMDB)'

### Using functions to help ease our workflow:

In [ ]:
def vaccum(url: str) -> BeautifulSoup:
    try:
        # I am not a robot lmao
        needed_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        res_func = requests.get(url,headers=needed_headers)

        # Raise exc if status code is 5xx or 4xx
        res_func.raise_for_status()

        return BeautifulSoup(res_func.text, "lxml")
    
    except requests.exceptions.RequestException as e:
        # Catch all request-related errors 
        print(f"An Error occured!: {e}")
        return None
    
    finally:
        print("Excecution FIN")

In [10]:
# test case 1: a working url
vaccumed = vaccum("https://www.themoviedb.org/movie")
# This works, won't call it for obvious reasons 

Excecution FIN


In [12]:
# test case 2: malformed/incorrect URLs
vaccum("https://www.themovipepepe.org/what")

An Error occured!: HTTPSConnectionPool(host='www.themovipepepe.org', port=443): Max retries exceeded with url: /what (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000025E58059CA0>: Failed to resolve 'www.themovipepepe.org' ([Errno 11001] getaddrinfo failed)"))
Excecution FIN


In [6]:
# Displaying the name of the first movie
first_movie = soup.select(".content h2")[3].getText()
first_movie

'Lilo & Stitch'

In [7]:
# Extracting User rating of the first movie
span_class = soup.select_one(".user_score_chart .icon")['class'][1]
span_class
perc1 = span_class.split("icon-r")[1]
print(f"User score of the movie: {first_movie} is {perc1}%")

User score of the movie: Lilo & Stitch is 71%


In [8]:
# gonna use regex for extracting the part of the url following the string “https://www.themoviedb.org/” 
text = str(soup.select(".content h2"))
match = re.search(r'movie/\d\d\d\d\d\d',text)
match.group()
# This is going to help us later (i think)

'movie/552524'

### Grabbing the title of all the movies

In [11]:
def title_grabber(soup: BeautifulSoup) -> list:
    titles = []
    movies = soup.select(".content h2")[3:]
    for movie in movies:
        a_tag = movie.find("a", title = True)
        if a_tag:
            titles.append(a_tag["title"])
    return titles
    
        
title_grabber(vaccumed)

['Lilo & Stitch',
 'Sinners',
 'Sikandar',
 'A Minecraft Movie',
 'Shadow Force',
 'A Working Man',
 'Final Destination Bloodlines',
 'Mission: Impossible - The Final Reckoning',
 'The Last Stand of Ellen Cole',
 'Until Dawn',
 'The Suicide Squad',
 'Warfare',
 'The Ugly Stepsister',
 'Captain America: Brave New World',
 'Lilo & Stitch',
 'The Containment',
 'Snow White',
 "A Widow's Game",
 'Tin Soldier',
 'Thunderbolts*']

### Grabbing the user ratings of all the movies in a page

In [12]:
def rating_grabber(soup: BeautifulSoup) -> list:
    user_score = []
    scores = soup.select(".content .user_score_chart")

    for score in scores:
        # edge case
        if score is None:
            user_score.append("No rating")
        else:
            percent = score["data-percent"]
            user_score.append(percent)
        
        

    return user_score
rating_grabber(vaccumed)

['71',
 '75',
 '52',
 '65',
 '62',
 '67',
 '70',
 '71',
 '82',
 '65',
 '75',
 '73',
 '70',
 '61',
 '76',
 '77',
 '43',
 '64',
 '50',
 '74']

In [13]:
soup.select(".content h2")

[<h2>Sort</h2>,
 <h2>Where To Watch <span>279</span></h2>,
 <h2>Filters</h2>,
 <h2><a href="/movie/552524-lilo-stitch" title="Lilo &amp; Stitch">Lilo &amp; Stitch</a></h2>,
 <h2><a href="/movie/1233413-sinners" title="Sinners">Sinners</a></h2>,
 <h2><a href="/movie/1257960" title="Sikandar">Sikandar</a></h2>,
 <h2><a href="/movie/950387-a-minecraft-movie" title="A Minecraft Movie">A Minecraft Movie</a></h2>,
 <h2><a href="/movie/757725-shadow-force" title="Shadow Force">Shadow Force</a></h2>,
 <h2><a href="/movie/1197306-a-working-man" title="A Working Man">A Working Man</a></h2>,
 <h2><a href="/movie/574475-final-destination-bloodlines" title="Final Destination Bloodlines">Final Destination Bloodlines</a></h2>,
 <h2><a href="/movie/575265-mission-impossible-the-final-reckoning" title="Mission: Impossible - The Final Reckoning">Mission: Impossible - The Final Reckoning</a></h2>,
 <h2><a href="/movie/1411773-the-last-stand-of-ellen-cole" title="The Last Stand of Ellen Cole">The Last Sta

### Extracting the HTML gold of all the individual pages of movies

In [14]:
def html_content(soup: BeautifulSoup) -> list:
    my_content = str(soup.select(".content h2"))
    my_list_of_contents = []

    stuff = re.findall(r"movie/\d\d\d\d\d\d", my_content)
    my_list_of_contents.extend(stuff)

    return my_list_of_contents
   
html_content(vaccumed)
    

['movie/552524',
 'movie/123341',
 'movie/125796',
 'movie/950387',
 'movie/757725',
 'movie/119730',
 'movie/574475',
 'movie/575265',
 'movie/141177',
 'movie/123254',
 'movie/436969',
 'movie/124143',
 'movie/128412',
 'movie/822119',
 'movie/568770',
 'movie/447273',
 'movie/139783',
 'movie/977294',
 'movie/986056']

In [15]:
soup.select(".multi_select")[-2]

<ul class="multi_select text" id="with_genres" name="with_genres[]">
<li data-value="28"><a class="no_click" href="/discover/movie?with_genres=28">Action</a></li>
<li data-value="12"><a class="no_click" href="/discover/movie?with_genres=12">Adventure</a></li>
<li data-value="16"><a class="no_click" href="/discover/movie?with_genres=16">Animation</a></li>
<li data-value="35"><a class="no_click" href="/discover/movie?with_genres=35">Comedy</a></li>
<li data-value="80"><a class="no_click" href="/discover/movie?with_genres=80">Crime</a></li>
<li data-value="99"><a class="no_click" href="/discover/movie?with_genres=99">Documentary</a></li>
<li data-value="18"><a class="no_click" href="/discover/movie?with_genres=18">Drama</a></li>
<li data-value="10751"><a class="no_click" href="/discover/movie?with_genres=10751">Family</a></li>
<li data-value="14"><a class="no_click" href="/discover/movie?with_genres=14">Fantasy</a></li>
<li data-value="36"><a class="no_click" href="/discover/movie?with_ge

### Grabbing the genres of the movies

In [16]:
def genre_grabber(soup: BeautifulSoup) -> list:
    genres = soup.select(".multi_select")[-2]
    my_genres = []
    for gen in genres.find_all("li"): # found them in list item (li) using inspect
        a_tag = gen.find("a") # because its in a hyerplink?
        if a_tag:
            my_genres.append(a_tag.text.strip())
    return my_genres
genre_grabber(vaccumed)
    

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [18]:
# ok so what I plan to do is that
# I already have a list of all the movie content IDs
# gonna iterate through them to create a new list object which will be my inputs for my cast func
html_content(vaccumed)
load = []
for info in html_content(vaccumed):
    load.append('https://www.themoviedb.org/' + info)
load

['https://www.themoviedb.org/movie/552524',
 'https://www.themoviedb.org/movie/123341',
 'https://www.themoviedb.org/movie/125796',
 'https://www.themoviedb.org/movie/950387',
 'https://www.themoviedb.org/movie/757725',
 'https://www.themoviedb.org/movie/119730',
 'https://www.themoviedb.org/movie/574475',
 'https://www.themoviedb.org/movie/575265',
 'https://www.themoviedb.org/movie/141177',
 'https://www.themoviedb.org/movie/123254',
 'https://www.themoviedb.org/movie/436969',
 'https://www.themoviedb.org/movie/124143',
 'https://www.themoviedb.org/movie/128412',
 'https://www.themoviedb.org/movie/822119',
 'https://www.themoviedb.org/movie/568770',
 'https://www.themoviedb.org/movie/447273',
 'https://www.themoviedb.org/movie/139783',
 'https://www.themoviedb.org/movie/977294',
 'https://www.themoviedb.org/movie/986056']

### Grabbing the casts of alll the movies

In [33]:
def cast_grabber(soup: BeautifulSoup) -> list: 

    cast_list = []

    def castisize():
        cards = soup.select("li.card")

        # this loop creates a beautifulsoup instance of every link request
        for card in cards:
              name = card.find("p").get_text(strip=True)
              print(name)
              cast_list.append(name)
        
    castisize()
    return cast_list  

# my epic use of hashmap 
all_casts = {}
for link in load:
     res = requests.get(link,headers=needed_headers)
     soup = BeautifulSoup(res.text,"lxml")
     all_casts[link] = cast_grabber(soup)



Maia Kealoha
Sydney Agudong
Chris Sanders
Zach Galifianakis
Billy Magnussen
Courtney B. Vance
Amy Hill
Tia Carrere
Kaipo Dudoit
Rie Nakagawa
Yuri Yamashina
Kiyoshi Abe
Chizuyu Azami
Moeko Ezawa
Toshihiko Oda
Hiroshi Chō
Akira Takahashi
Ikunosuke Koizumi
Pascal Obispo
Jason Momoa
Jack Black
Sebastian Eugene Hansen
Emma Myers
Danielle Brooks
Jennifer Coolidge
Rachel House
Allan Henry
Bram Scott-Breheny
Kerry Washington
Omar Sy
Jahleel Kamara
Mark Strong
Da'Vine Joy Randolph
Method Man
Ed Quinn
Natalia Reyes
Marvin Jones III
Kaitlyn Santa Juana
Teo Briones
Rya Kihlstedt
Richard Harmon
Owen Patrick Joyner
Anna Lore
Brec Bassinger
Tony Todd
Andrew Tinpo Lee
Tom Cruise
Hayley Atwell
Ving Rhames
Simon Pegg
Esai Morales
Pom Klementieff
Henry Czerny
Holt McCallany
Janet McTeer
Sergio Ramos
Carmen Salinas
Rosalinda Risso
Lorena Herrera
María Montaño
Pilar Delgado
Vitorino
Marco Antonio Sánchez
Paco Sañudo
Margot Robbie
Idris Elba
John Cena
Joel Kinnaman
Sylvester Stallone
Viola Davis
David Dastm

In [24]:
# iteratin through the items of the dict to get clean result
for url, cast in all_casts.items():
     print(f"Cast for {url} -> {cast}")

Cast for https://www.themoviedb.org/movie/950387 -> ['Jason Momoa', 'Jack Black', 'Sebastian Eugene Hansen', 'Emma Myers', 'Danielle Brooks', 'Jennifer Coolidge', 'Rachel House', 'Allan Henry', 'Bram Scott-Breheny']
Cast for https://www.themoviedb.org/movie/574475 -> ['Kaitlyn Santa Juana', 'Teo Briones', 'Rya Kihlstedt', 'Richard Harmon', 'Owen Patrick Joyner', 'Anna Lore', 'Brec Bassinger', 'Tony Todd', 'Andrew Tinpo Lee']
Cast for https://www.themoviedb.org/movie/447273 -> ['Rachel Zegler', 'Gal Gadot', 'Andrew Burnap', 'Jeremy Swift', 'Jason Kravits', 'Martin Klebba', 'George Salazar', 'Tituss Burgess', 'Andy Grotelueschen']
Cast for https://www.themoviedb.org/movie/119730 -> []
Cast for https://www.themoviedb.org/movie/124143 -> []
Cast for https://www.themoviedb.org/movie/148079 -> []
Cast for https://www.themoviedb.org/movie/977294 -> ['Scott Eastwood', 'Robert De Niro', 'Jamie Foxx', 'John Leguizamo', 'Nora Arnezeder', 'Shamier Anderson', 'Yul Vazquez', 'Rita Ora', 'Saïd Taghma

NameError: name 'cast_list' is not defined

In [ ]:
# next need a user defined function that returns a pandas data frame with the above data
def bamboo():
    mydata = {"Title" : title_grabber(vaccumed), "User ratings" : rating_grabber(vaccumed), "Genre" : genre_grabber(vaccumed)}
    df = pd.DataFrame(data=mydata)
    return df

bamboo()
    
    
    

In [47]:
len(title_grabber(vaccumed))


20

In [44]:
len(rating_grabber(vaccumed))

20

In [48]:
len(genre_grabber(vaccumed))

19

0